In [ ]:
# !pip install -U pip
# !pip install tensorflow-gpu==2.0.0-rc0
# !pip install matplotlib

In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import math
import datetime
import os
from util import Recipes
import random
print(tf.__version__)

2.0.0


In [2]:
r = Recipes()

(1035530,) (32027,)


In [3]:
g = r.batch_iterator(r.train_recipes, 200)
x, y = g.__next__()
i = random.randint(0, 199)
print(r.onehot_to_hr_recipe(x[i]))
print()
print(r.onehot_to_hr_recipe(y[i]))

(18, ['salt', 'pepper', 'garlic', 'onion', 'olive oil', 'olive', 'black pepper', 'cream', 'lemon', 'chicken', 'clove', 'thyme', 'virgin olive oil', 'chicken stock', 'chives', 'white pepper', 'minced onion', 'squash'])

(4, ['lemon juice', 'sour cream', 'extra virgin olive oil', 'apples'])


In [4]:
batch_size = 256
valid_batch_size = 512
epochs = 1

model = tf.keras.models.Sequential([tf.keras.layers.Dense(500, input_shape=(3500,)),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(330),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(300),
                                    tf.keras.layers.LeakyReLU(),
                                                                                                            
                                    tf.keras.layers.Dense(330),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(500),
                                    tf.keras.layers.LeakyReLU(),
                                    
                                    tf.keras.layers.Dense(len(r.ingredients), activation='sigmoid')
                                   ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, update_freq=batch_size*64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               1750500   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 330)               165330    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 330)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               99300     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 330)               9

**note: up to ~0.43 validation categorical_accuracy is possible with the right architecture and by training for 4-8 epochs**

In [ ]:
history = model.fit_generator(r.batch_iterator(r.train_recipes, batch_size=batch_size), 
                              epochs=epochs, 
                              steps_per_epoch=r.get_batch_count(r.train_recipes, batch_size), 
                              validation_data=r.batch_iterator(r.test_recipes, batch_size=valid_batch_size), 
                              validation_steps=r.get_batch_count(r.test_recipes, valid_batch_size),
                              #callbacks=[tensorboard_callback]
                             )

 980/4045 [======>.......................] - ETA: 4:34 - loss: 11.9574 - categorical_accuracy: 0.1012

In [ ]:
tf.saved_model.save(model, "recipenet_saved_model/1/")

In [7]:
def no_overlap(item, s):
    for k in s:
        if item in k or k in item:
            return False
    return True

In [8]:
r.match_ingr('tortilla')

['tortillas',
 'a',
 'flour tortillas',
 'corn tortillas',
 'tortilla chips',
 'corn tortilla chips',
 'flour tortilla',
 'baked corn tortilla chips',
 'tortilla shells',
 'tortilla wraps']

In [24]:
i = r.labels_to_batch(['sushi rice', 'rice', 'cucumber', 'sugar', 'nori', 'salt'])
#i = r.labels_to_batch(['rice', 'black beans', 'tortillas', 'flour tortillas', 'guacamole', 'peppers'])
#i = r.labels_to_batch(['rice', 'oil', 'salt', 'pepper', 'spices', 'seasoning', 'vegetables', 'carrots', 'bell peppers', 'tomatoes'])
#i = r.labels_to_batch(['butter', 'sugar', 'eggs', 'vanilla', 'flour', 'blueberries'])
p = model.predict_on_batch(i)
_, hr = r.onehot_to_hr_recipe(i[0]) 

print(hr, '-->\n')

n = 16
added_ingr = list(reversed(r.ingredients[np.argsort(p[0])[-n:]]))
added_ingr_p = list(reversed(p[0].numpy()[np.argsort(p[0])[-n:]]))

n_c = 0
for ingr, p in zip(added_ingr, added_ingr_p):
    no_overlap_ = no_overlap(ingr, set(hr))
    print(('' if no_overlap_ else '(')+ingr+('' if no_overlap_ else ')'), '|', str(p)[:5], '|', '.')
    #if no_overlap_: n_c += 1
    #if n_c > 6: break

['salt', 'sugar', 'rice', 'cucumber', 'nori', 'sushi rice'] -->

(sushi rice) | 0.109 | .
seaweed | 0.069 | .
vinegar | 0.044 | .
radish | 0.033 | .
water | 0.029 | .
wasabi | 0.026 | .
dashi | 0.015 | .
mirin | 0.015 | .
(nori) | 0.015 | .
sake | 0.014 | .
black sesame seeds | 0.011 | .
tofu | 0.010 | .
(cucumber) | 0.009 | .
sliced | 0.006 | .
scallions | 0.006 | .
(cucumbers) | 0.006 | .


In [25]:
g = r.batch_iterator(r.test_recipes, 128)
x, y = g.__next__()
p = model.predict_on_batch(x)
np.sum(p)

284.8895